# LlamaIndex Dataset Creation

In [16]:
from pymupdf4llm import LlamaMarkdownReader

# Define the reader object
llama_reader = LlamaMarkdownReader()

# Load the data
indexed_chunks = llama_reader.load_data("../data/raw/sample.pdf")

Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing ../data/raw/sample.pdf...
[                                        ] (0/1=======================================[================

# LLM Finetuning Dataset Creation

In [81]:
from openai import OpenAI


class LLM:
    def __init__(self, client: OpenAI, n_questions: int = 3):
        self.client = client
        self.n_questions = n_questions

    def generate_questions_and_answers(self, indexed_chunks):
        import random

        responses = list()

        for i in range(2):
            # Select a random temperature between 0.0 and 0.2
            temperature = round(random.uniform(0.0, 0.2), 2)

            # Define the prompt
            completion = self.client.chat.completions.create(
                model="bartowski/Llama-3-8B-Instruct-Gradient-1048k-GGUF",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that can generate questions and answers from the provided text.",
                    },
                    {
                        "role": "user",
                        "content": f"Generate {self.n_questions} questions and answers from the provided text. Output should only include the questions and answers. There must not be any other text, dash, line, or output.",
                    },
                    {"role": "user", "content": indexed_chunks[i].text},
                ],
                temperature=temperature,
                response_format={"type": "json_object"},
            )

            # Get the response
            response = completion.choices[0].message.content.strip("</s>")

            # Add the response to the list
            responses.append(response)

        return responses

    def convert_to_json(self, questions_and_answers):
        import json

        responses = list()

        for i in range(len(questions_and_answers)):
            # Define the prompt
            completion = self.client.chat.completions.create(
                model="bartowski/Llama-3-8B-Instruct-Gradient-1048k-GGUF",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that convert the provided text to valid JSON.",
                    },
                    {
                        "role": "user",
                        "content": "Convert the provided text to valid JSON. Output should only be the valid JSON. For example: {[{'question': question1, 'answer': answer1}, {'question': question2, 'answer': answer2}, {'question': question3, 'answer': answer3}]}. There must not be any other text, dash, line, or output.",
                    },
                    {"role": "user", "content": questions_and_answers[i]},
                ],
                temperature=0.0,
                response_format={"type": "json_object"},
            )

            # Get the response
            response = completion.choices[0].message.content.strip("</s>")

            # Convert the response to a JSON object
            response_json = json.loads(response)

            # Add the response to the list
            responses.append(response_json)

        return responses

    def generate_dataset(self, indexed_chunks):
        response = self.generate_questions_and_answers(indexed_chunks)
        response_json = self.convert_to_json(response)

        return response_json

In [82]:
# Define the OpenAI client
client = OpenAI(base_url="http://172.29.3.249:1234/v1", api_key="lm-studio")

# Define the LLM object
llm = LLM(client=client)

# Generate the dataset
response = llm.generate_dataset(indexed_chunks)
response

2


[[{'question': 'What is multicriteria decision making and how does it differ from single-criterion decision making?',
   'answer': 'Multicriteria decision making involves screening, prioritizing, ranking or selecting alternatives based on human judgment from among a finite set of decision alternatives in terms of multiple usually conflicting criteria. This differs from single-criterion decision making which only considers one criterion.'},
  {'question': 'What is the significance of weights in MCDM models?',
   'answer': 'Weights play a very significant role in MCDM models as they provide information about the relative importance of the considered criteria. Different methods are developed to take these priorities into account, and using ranks to elicit weights by some formulas is more reliable than just directly assigning weights to criteria.'},
  {'question': 'What are some examples of situations where rank ordering weight methods can be useful?',
   'answer': 'Rank ordering weight me